<a href="https://colab.research.google.com/github/Gustavo-br-rj/PROJ-BI_MASTER/blob/main/4_PREVER_TEMP_MANCAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Previsão de Série Temporal da Temperatura de Mancais Utilizando LSTM


## Parte 0 - Importação das Bibliotecas

In [2]:
#%tensorflow_version 1.x
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import math
import json

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

#tf.__version__

In [18]:
def prepareSetRN(windowSize, outputSize, flatSet ):
  preparedX = []
  preparedY = []
  setSize = len(flatSet)
  for i in range(windowSize, setSize - (outputSize-1)):
    preparedX.append(flatSet[i-windowSize:i, 0])
    preparedY.append(flatSet[i:i + outputSize, 0])

  preparedX, preparedY = np.array(preparedX), np.array(preparedY)
  return preparedX, preparedY

## Parte 1 - Importação do Dataset

In [3]:
# Importação da Base de Dados
dataset = pd.read_csv('/content/drive/MyDrive/BI_MASTER/PROJETO_FINAL_BIMASTER/DATASETS/dataset_C-4451_08001A.csv')

In [4]:
#Conversão do campo data para datetime e ordenação do dataset
dataset['DATA'] = pd.to_datetime(dataset['DATA'],dayfirst=True)
dataset = dataset.sort_values(['EQUIPAMENTO', 'MANCAL', 'DATA'], ascending=[1, 1, 1])

In [19]:
#Diretório dos modelos de RN para cada mancal
dirModel = '/content/drive/MyDrive/BI_MASTER/PROJETO_FINAL_BIMASTER/MODELOS/'

#Buscando a identificação dos mancais
mancais = dataset['MANCAL'].unique().tolist()

for i in range(len(mancais)):
  manc = dataset.loc[(dataset["MANCAL"] == mancais[i]), ["EQUIPAMENTO","MANCAL","DATA","TEMPERATURA"]]
  manc = manc.reset_index(drop=True)

  nomeModelo  = '{0}{1}__{2}_model'.format(dirModel,manc['EQUIPAMENTO'].to_numpy()[0], manc['MANCAL'].to_numpy()[0])
  paramModelo = '{0}{1}__{2}_params.json'.format(dirModel,manc['EQUIPAMENTO'].to_numpy()[0], manc['MANCAL'].to_numpy()[0])

  #Obtendo o arquivo de parâmetros
  f_json = open(paramModelo,)
  json_param = json.load(f_json)
  f_json.close()

  #Reconstruindo o modelo
  reconstructed_model = load_model(nomeModelo)

  #Pegando a última janela de dados do conjunto
  window = json_param["window"]
  last_window_set = manc.tail(window)
  last_window_set = last_window_set.iloc[0:window, 3:4].values
  
  #Normalizando
  sc = MinMaxScaler(feature_range = (0, 1))
  last_window_set_scaled = sc.fit_transform(last_window_set)

  output = json_param["output"]
  #Prever temperatura
  X_last_window = []
  y_last_window = []
  X_last_window, y_last_window = prepareSetRN(window, output, last_window_set_scaled)
  X_last_window = np.reshape(X_last_window, (X_last_window.shape[0], X_last_window.shape[1], 1))

  predicted = reconstructed_model.predict(X_last_window)
  predicted = sc.inverse_transform(predicted)

  break

predicted 

IndexError: ignored

## Parte 2 - Preprocessamento de Dados

### Criar a estrutura de dados com janela WINDOW e output OUTPUTSIZE para entrada na RN

In [ ]:
def prepareSetRN(windowSize, outputSize, flatSet ):
  preparedX = []
  preparedY = []
  setSize = len(flatSet)
  for i in range(windowSize, setSize - (outputSize-1)):
    preparedX.append(flatSet[i-windowSize:i, 0])
    preparedY.append(flatSet[i:i + outputSize, 0])

  preparedX, preparedY = np.array(preparedX), np.array(preparedY)
  return preparedX, preparedY

### Construção da Rede completa

In [ ]:
def backToFlat( windowSet ):
    flatSet     = []
    window_size = windowSet.shape[1]
    setSize     = len(windowSet)

    #Adiciona as primeiras colunas do primeiro registro quando a janela > 1
    if window_size > 1:
      for i in range(0, window_size-1):
        flatSet.append(windowSet[0, i])

    for i in range(0, setSize):
        flatSet.append(windowSet[i, window_size-1])

    flatSet = np.array(flatSet)

    return np.reshape(flatSet, (flatSet.shape[0], 1)) 